In [1]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general
import scqubits as sc


In [27]:
#Validating T2 Calcs Against SCQUBITS

EJ = torch.tensor(14, requires_grad=True, dtype=torch.double)
EC = torch.tensor(4.6, requires_grad=True, dtype=torch.double)
EL = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
flux = torch.tensor(0.5, requires_grad=True, dtype=torch.double)
dim = 110


fluxonium_auto = Fluxonium(EJ, EC, EL, flux, dim, 'auto_H')
fluxonium_create = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')
fluxonium_sc = sc.Fluxonium(EJ = EJ.item(), EC = EC.item(), EL = EL.item(), flux = flux.item(),cutoff = dim)

print((fluxonium_sc.t1_quasiparticle_tunneling()*general.effective_t1_rate(fluxonium_create, "t1_quasiparticle_tunneling").item()))
print((fluxonium_sc.t1_charge_impedance()*general.effective_t1_rate(fluxonium_auto , "t1_charge_impedance").item()))
print((fluxonium_sc.t1_capacitive()*general.effective_t1_rate(fluxonium_auto, "t1_capacitive").item()))
print((fluxonium_sc.t1_inductive() * general.effective_t1_rate(fluxonium_auto, "t1_inductive").item()))
print((fluxonium_sc.t1_flux_bias_line() * general.effective_t1_rate(fluxonium_auto , "t1_flux_bias_line").item()))
print((fluxonium_sc.tphi_1_over_f_cc() * general.effective_tphi_rate(fluxonium_auto , "tphi_1_over_f_cc").item()))
print((fluxonium_sc.tphi_1_over_f_flux() * general.effective_tphi_rate(fluxonium_auto, "tphi_1_over_f_flux()").item()))



2625494610514317.0
0.8974570212084598
0.897457005239348
0.897456885356036
0.8974570052393152
1.0000000000000226
6509041.996963719


In [3]:
general.t2_rate(fluxonium_create, fluxonium_create.t1_supported_noise_channels(), fluxonium_create.tphi_supported_noise_channels())


tensor([[0.0027]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [16]:
#Optimisation

#FIXED PARAMS
dim = 110
learn_rate = 1
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EC_bounds = [1e-3, 8]
EL_bounds = [2e-1, 10]

#RECORDING VALUES
EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
R2_rate = [1e100, 1e99]

# GRADIENT DESCENT
i = 0
while R2_rate[-1] < R2_rate[-2]:
    i+=1

    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')
    R2  = general.t2_rate(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())
    R2_rate.append(R2.item())

    R2.backward()
    with torch.no_grad():
        #If paramater value drops below lower bound, it will remain on the lower bound
        ##WHY DO WE - the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        EC.data = EC - EC.grad*learn_rate if EC - EC.grad*learn_rate > EC_bounds[0] else torch.tensor(EC_bounds[0])
    
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EC_vals.append(EC.item())
    EL_vals.append(EL.item())

    print(f"{i}: R2: {R2.item()}, EJ: {EJ.item()}, EL: {EL.item()}, EC: {EC.item()}")

1: T2: 8.094098612878955e-09, EJ: 16.52220159140853, EL: 3.8741305318935844, EC: 4.60487772526564
2: T2: 8.280278900637147e-09, EJ: 16.534038969477393, EL: 3.841330067404866, EC: 4.5816676722827605
3: T2: 8.666630362840458e-09, EJ: 16.55153162040369, EL: 3.7927995232917233, EC: 4.547237618986515
4: T2: 9.282946426753768e-09, EJ: 16.574375894135947, EL: 3.7293156122381133, EC: 4.5020400383567045
5: T2: 1.0178772273252253e-08, EJ: 16.602187190267465, EL: 3.651868746884678, EC: 4.446661765941949
6: T2: 1.1430213238358809e-08, EJ: 16.634516302729388, EL: 3.561625664544865, EC: 4.381809002042788
7: T2: 1.315087379150966e-08, EJ: 16.670868457123493, EL: 3.459884837346616, EC: 4.308287826381523
8: T2: 1.5509017083054253e-08, EJ: 16.710724001812224, EL: 3.3480265898165946, EC: 4.226980256857842
9: T2: 1.8754288201244953e-08, EJ: 16.753559492689185, EL: 3.227460498668116, EC: 4.138816314881693
10: T2: 2.3259277574767162e-08, EJ: 16.79886777246708, EL: 3.0995732130157627, EC: 4.044743110321568
1